In [ ]:
# initialize model and print out the statistics yaml
import json
from IPython.display import display, Markdown
import yaml

import torch.nn as nn

from mymodel import init_my_model


model = init_my_model() 
min_max_yaml = yaml.safe_load(open("min_max.yaml", "r"))
config = json.load(open("config.json", "r"))
stats = yaml.safe_load(open("stats.yaml", "r"))

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

model_name = model.__class__.__name__
num_layers = len(list(model.net.children()))
num_epochs = stats["epochs"]
best_train_loss = stats["best_train_loss"]
best_test_loss = stats["best_test_loss"]
best_val_loss = stats["best_val_loss"]

mae_u_train = stats["metrics"]["train"]["u"]["mae"]
mae_v_train = stats["metrics"]["train"]["v"]["mae"]

mae_u_test = stats["metrics"]["test"]["u"]["mae"]
mae_v_test = stats["metrics"]["test"]["v"]["mae"]

mae_u_val = stats["metrics"]["validation"]["u"]["mae"]
mae_v_val = stats["metrics"]["validation"]["v"]["mae"]

rmse_u_train = stats['metrics']['train']['u']['rmse']
rmse_v_train = stats['metrics']['train']['v']['rmse']

rmse_u_test = stats['metrics']['test']['u']['rmse']
rmse_v_test = stats['metrics']['test']['v']['rmse']

rmse_u_val = stats['metrics']['validation']['u']['rmse']
rmse_v_val = stats['metrics']['validation']['v']['rmse']


In [ ]:
md_text = "# Model Architecture\n"
md_text += f"The following section describes the architecture of the {model_name} used for the submission. "
md_text += f"The {model_name} has a total number of {total_params} parameters "
md_text += f"with {trainable_params} trainable parameters. "
md_text += f"It consists of {num_layers} layers. "
md_text += "The layers are structured as follows:\n\n"
md_text += "| Index | Layer Type | Details |\n"
md_text += "| :--- | :--- | :--- |\n"

for i, layer in enumerate(model.net.children()):
    layer_type = layer.__class__.__name__
    
    if isinstance(layer, nn.Conv2d):
        details = f"In: {layer.in_channels}, Out: {layer.out_channels}, Kernel: {layer.kernel_size[0]}x{layer.kernel_size[1]}"
    elif isinstance(layer, nn.LeakyReLU):
        details = f"Negative Slope: {layer.negative_slope}"
    else:
        details = "---"
        
    md_text += f"| {i} | {layer_type} | {details} |\n"

display(Markdown(md_text))

In [ ]:
md_text = "This was the configuration used to initialize the model and load the data:\n\n"
md_text += "| Parameter | Value |\n"
md_text += "| :--- | :--- |\n"

for k, v in config.items():
    display_key = k.replace("_", " ").title()
    md_text += f"| {display_key} | {v} |\n"

display(Markdown(md_text))

In [ ]:
def format_dict_to_md(d, level=0):
    lines = []
    for k, v in d.items():
        indent = "  " * level
        if isinstance(v, dict) and v.get("min") or v.get("max"):
            lines.append(f"{indent}* **{k.capitalize()}**: From {v['min']:.4f} to {v['max']:.4f}")
        elif isinstance(v, dict):
            lines.append(f"{indent}* **{k.capitalize()}**:")
            lines.extend(format_dict_to_md(v, level + 1))
        else:
            lines.append(f"{indent}* **{k}**: {v}")
    return lines

md_text = "The input data was then normalized between 0 and 1 based on those ranges:\n\n"
md_text += "\n".join(format_dict_to_md(min_max_yaml))
display(Markdown(md_text))

In [ ]:
md_text = "# Evaluation\n"
md_text += "## Loss Graphs\n"
md_text += f"The following figure shows the training, validation and test loss over the {num_epochs} epochs the model was trained on.\n\n"
md_text += "![Train and test losses](losses.png)\n\n"
md_text += f"The best losses archieved while training where {best_train_loss:.4g} on the training data, "
md_text += f"{best_test_loss:.4g} on the test data and {best_val_loss:.4g} on the validation data.\n\n"
md_text += "## Performance Metrics\n"
md_text += "The table below summarizes the error metrics for the velocity components u and v across all data splits.\n\n"
md_text += "| Metric | Field | Training | Test | Validation |\n"
md_text += "| :--- | :--- | :--- | :--- | :--- |\n"
md_text += f"| **Mean absolute error** | u | {mae_u_train:.3g} | {mae_u_test:.3g} | {mae_u_val:.3g} |\n"
md_text += f"| | v | {mae_v_train:.3g} | {mae_v_test:.3g} | {mae_v_val:.3g} |\n"
md_text += f"| **Root mean square error** | u | {rmse_u_train:.3g} | {rmse_u_test:.3g} | {rmse_u_val:.3g} |\n"
md_text += f"| | v | {rmse_v_train:.3g} | {rmse_v_test:.3g} | {rmse_v_val:.3g} |\n"

display(Markdown(md_text))

# Predictions

## Interpolation

Sample data point predictions and comparisons to their labels (and visualization of vector fields)


In [1]:
from pathlib import Path
submission_dir = Path(globals()['_dh'][0])

plot_dir = submission_dir / "plots" / "interpolation"

md_text = ""

error_plots = []
quiver_plots = []
prediction_plots = []

for plot in plot_dir.iterdir():
    if plot.is_file():
        if plot.name.startswith("error"):
            error_plots.append(plot)
        elif plot.name.endswith("quiver.png"):
            quiver_plots.append(plot)
        elif plot.name.startswith("prediction"):
            prediction_plots.append(plot)

for pred, quiver, error in zip(prediction_plots, quiver_plots, error_plots):
    flow_value = pred.name.split("_")[-1].split(".png")[0]
    md_text += f"![{flow_value} Quiver](plots/interpolation/{quiver.name})\n\n"
    md_text += f"![{flow_value} Prediction](plots/interpolation/{pred.name})\n\n"
    md_text += f"![{flow_value} Error](plots/interpolation/{error.name})\n\n"

display(Markdown(md_text))

FileNotFoundError: [Errno 2] No such file or directory: '/home/bene/CLionProjects/numsim/resources/plots/interpolation'

## Extrapolation

To see how well the model generalizes, the following section should contain three out-of-distribution test samples.

In [ ]:
from pathlib import Path
submission_dir = Path(globals()['_dh'][0])

plot_dir = submission_dir / "plots" / "extrapolation"

md_text = ""

error_plots = []
quiver_plots = []
prediction_plots = []

for plot in plot_dir.iterdir():
    if plot.is_file():
        if plot.name.startswith("error"):
            error_plots.append(plot)
        if plot.name.endswith("quiver.png"):
            quiver_plots.append(plot)
        elif plot.name.startswith("prediction"):
            prediction_plots.append(plot)

for pred, quiver in zip(prediction_plots, quiver_plots):
    flow_value = pred.name.split("_")[-1].split(".png")[0]
    md_text += f"![{flow_value} Quiver](plots/extrapolation/{quiver.name})\n\n"
    md_text += f"![{flow_value} Prediction](plots/extrapolation/{pred.name})\n\n"

display(Markdown(md_text))